In [1]:
!pip install transformers datasets

In [3]:
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
import json

# Load dữ liệu
with open("./vietnamese_news_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Chuyển dữ liệu sang dạng HuggingFace Dataset
train_data = [
    {"text": item["text"], "summary": item["summary"]} for item in data
]
dataset = Dataset.from_list(train_data)

# Load tokenizer và model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


# Tiền xử lý dữ liệu
def preprocess(example):
    inputs = tokenizer(example["text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example["summary"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs


tokenized_dataset = dataset.map(preprocess, remove_columns=["text", "summary"])

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Huấn luyện
trainer.train()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss
10,3.206700
20,2.429600
30,1.906900
40,1.559700
50,1.700400
60,1.350100
70,1.423000
80,1.362200
90,1.226000
100,1.074400


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=150, training_loss=1.4801757176717123, metrics={'train_runtime': 132.2655, 'train_samples_per_second': 2.268, 'train_steps_per_second': 1.134, 'total_flos': 325065690316800.0, 'train_loss': 1.4801757176717123, 'epoch': 3.0})

In [10]:
tokenizer.save_pretrained("./bart_finetuned")
model.save_pretrained("./bart_finetuned")

In [13]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load lại mô hình và tokenizer đã fine-tune
model_name_or_path = "./bart_finetuned"
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
model = BartForConditionalGeneration.from_pretrained(model_name_or_path)


def summarize(text, max_length=128, min_length=30):
    inputs = tokenizer(
        [text], max_length=512, truncation=True, return_tensors="pt"
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


if __name__ == "__main__":
    text = "Hội nghị Trung ương 4 của Trung Quốc diễn ra tuần tới, với sự tham gia của Chủ tịch Tập Cận Bình và các lãnh đạo hàng đầu nước này. Từ thập niên 50, họ duy trì tổ chức hội nghị 5 năm một lần, để đưa ra các mục tiêu về kinh tế - xã hội. Năm nay, Bắc Kinh từng nhiều lần nhấn mạnh các thành tựu đạt được trong kế hoạch 5 năm lần thứ 14, bất chấp áp lực ngày càng lớn từ Mỹ trong lĩnh vực công nghệ và thương mại. Nước này sẽ vạch ra kế hoạch 5 năm lần thứ 15, với các mục tiêu dài hạn đến 2035 là nâng GDP bình quân lên mức trung bình của các nước phát triển, đồng thời đạt được \"bước đột phá lớn\" trong công nghệ lõi vào 2035. Tuy nhiên, thách thức trước mắt sẽ được các nhà hoạch định chính sách thảo luận tuần tới gồm đối phó việc Mỹ siết tiếp cận công nghệ, tăng thuế nhập khẩu thêm 100% với hàng Trung Quốc, và nhu cầu nội địa yếu. Chỉ số giá tiêu dùng (CPI) nước này tháng 9 giảm 0,3% so với cùng kỳ năm ngoái, mạnh hơn dự kiến, khiến họ chưa thể thoát giảm phát. Zong Liang - cựu trưởng nhóm nghiên cứu tại Bank of China dự báo thay vì công bố những thay đổi lớn, Bắc Kinh nhiều khả năng tập trung vào \"cải cách\" để thúc đẩy nhu cầu trong nước, từ giảm các rào cản kinh doanh giữa các địa phương đến mở rộng cấp visa. Họ cũng có thể cam kết hỗ trợ thêm cho lĩnh vực bán dẫn, trí tuệ nhân tạo và robot hình người. Toàn bộ chi tiết sẽ được công bố vào kỳ họp quốc hội tháng 3/2026, nhưng bản tóm tắt sau cuộc họp tuần tới có thể hé lộ sớm các ưu tiên. Tianchen Xu, nhà kinh tế học cấp cao khu vực châu Á của Economist Intelligence Unit nhận định: \"Phần công nghệ trong kế hoạch có thể tập trung vào hai lĩnh vực: AI và nghiên cứu khoa học cơ bản\". Giới chức Trung Quốc đặt mục tiêu tỷ lệ ứng dụng AI trong các lĩnh vực chủ chốt đạt 90% năm 2030. Trong khi Mỹ hạ cam kết về khí hậu, Trung Quốc lại có 5 năm tới để thực hiện lời hứa bắt đầu giảm lượng phát thải carbon sau năm 2030. Đến nay, Bắc Kinh vẫn tụt hậu trong việc đạt được mục tiêu giảm tỷ lệ phát thải. \"Chúng tôi cho rằng việc này vẫn khả thi, nhưng cần cải cách mạnh hơn thị trường điện và loại bỏ các ngành công nghiệp phát thải cao\", ông nói. Giới phân tích hiện chưa rõ liệu Trung Quốc có thể đạt các đột phá công nghệ và tăng trưởng kinh tế chỉ dựa vào lập kế hoạch hay không. Larry Hu - kinh tế trưởng phụ trách Trung Quốc tại Macquarie cho biết 3 thay đổi kinh tế quan trọng nhất trong 25 năm qua của nước này \"được thúc đẩy bởi sự thay đổi nhu cầu bên ngoài hơn là các kế hoạch 5 năm\". Đó là vai trò ngày càng tăng của Trung Quốc trong chuỗi cung ứng toàn cầu sau khi gia nhập Tổ chức Thương mại Thế giới (WTO) và quyết định siết cho vay bất động sản của Bắc Kinh. Một bước ngoặt lớn khác là xuất khẩu được dự báo chững lại, sau giai đoạn chống chịu tốt trước các mức thuế của Mỹ. Bên cạnh đó, tham vọng về xe điện của nền kinh tế thứ hai thế giới đang làm chao đảo ngành công nghiệp ôtô toàn cầu. Tuy nhiên, Bắc Kinh cũng chưa đạt được một số mục tiêu trong sáng kiến \"Made in China 2025\" công bố từ 2015. Ví dụ, máy bay chở khách C919 do nước này sản xuất vẫn phụ thuộc vào linh kiện của Mỹ và châu Âu. Giới chức cũng đang lo ngại về cuộc chiến giá hình thành do dư thừa sản xuất trong bối cảnh nhu cầu nội địa yếu. Việc này khiến hàng giá rẻ của họ tràn ngập thế giới, gây áp lực lên nền công nghiệp của các quốc gia khác. Do đó, không chỉ Mỹ mà Mexico và Liên minh châu Âu (EU) cũng đe dọa áp thuế nhập khẩu. Bắc Kinh cho rằng các quốc gia không nên đổ lỗi cho Trung Quốc, mà chia sẻ công nghệ để phát huy thế mạnh riêng. Tháng trước, một số nước trong đó có Nga, đã họp tại Thiên Tân. Các nước này bày tỏ sự quan tâm tới việc áp dụng công nghệ của Trung Quốc, vốn thường rẻ hơn so với giải pháp của Mỹ hoặc châu Âu. Ví dụ, DeepSeek có lợi thế hơn OpenAI về chi phí vận hành và khả năng tiếp cận. Tuy nhiên, số khác lại coi sự trỗi dậy của Trung Quốc là mối đe dọa. Washington gần đây thắt chặt kiểm soát xuất khẩu và tăng thuế nhập khẩu khi Bắc Kinh liên tiếp đe dọa sự thống trị của Mỹ trong các ngành công nghiệp chủ chốt. Nền kinh tế lớn thứ hai thế giới đang đối mặt với thế tiến thoái lưỡng nan, khi phải cân bằng giữa an ninh quốc gia và tăng trưởng kinh tế. Họ cũng ưu tiên sự ổn định trong nước. Cuộc họp tuần tới sẽ hé lộ cách lãnh đạo Trung Quốc kết hợp các mục tiêu tưởng chừng mâu thuẫn này. Giới chức cũng kỳ vọng về khả năng ông Tập gặp Tổng thống Mỹ Donald Trump cuối tháng này tại Hàn Quốc, để giải quyết căng thẳng thương mại tái bùng phát gần đây. Hà Thu (theo Reuters, CNBC)"
    print("Tóm tắt:", summarize(text))


Tóm tắt: Trung Quốc diễn ra tuần tới, để đưa ra các mục tiêu về kinh tế - xã hội, nhằm lực công nghệ và thương mại.


In [4]:
!zip -r bart_finetuned.zip bart_finetuned

  adding: bart_finetuned/ (stored 0%)
  adding: bart_finetuned/checkpoint-100/ (stored 0%)
  adding: bart_finetuned/checkpoint-100/trainer_state.json (deflated 70%)
  adding: bart_finetuned/checkpoint-100/rng_state.pth (deflated 26%)
  adding: bart_finetuned/checkpoint-100/model.safetensors (deflated 7%)
  adding: bart_finetuned/checkpoint-100/training_args.bin (deflated 53%)
  adding: bart_finetuned/checkpoint-100/generation_config.json (deflated 47%)
  adding: bart_finetuned/checkpoint-100/optimizer.pt (deflated 8%)
  adding: bart_finetuned/checkpoint-100/config.json (deflated 62%)
  adding: bart_finetuned/checkpoint-100/scheduler.pt (deflated 61%)
  adding: bart_finetuned/checkpoint-100/scaler.pt (deflated 64%)
  adding: bart_finetuned/checkpoint-150/ (stored 0%)
  adding: bart_finetuned/checkpoint-150/trainer_state.json (deflated 73%)
  adding: bart_finetuned/checkpoint-150/rng_state.pth (deflated 26%)
  adding: bart_finetuned/checkpoint-150/model.safetensors (deflated 7%)
  adding